# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Loading-required-modules" data-toc-modified-id="Loading-required-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading required modules</a></div><div class="lev1 toc-item"><a href="#Memory-Network" data-toc-modified-id="Memory-Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Memory Network</a></div><div class="lev2 toc-item"><a href="#Debugging" data-toc-modified-id="Debugging-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Debugging</a></div><div class="lev2 toc-item"><a href="#Implementation-of-the-Loss-Function" data-toc-modified-id="Implementation-of-the-Loss-Function-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Implementation of the Loss Function</a></div><div class="lev1 toc-item"><a href="#Training-the-model" data-toc-modified-id="Training-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Training the model</a></div>

# Loading required modules

In [1]:
require 'nn';
require 'nngraph';

In [2]:
data_loader = require 'utils.data_loader';
QUICKLOAD = false

if QUICKLOAD then
    x = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/sample.t7")
    y = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/label.t7")
    voc = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7")
    index = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index")
else 
    input_file = "/Users/david/Documents/MemoryNetwork/preprocessing/output.txt"
    out_vocab_file = "/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7"
    out_tensor_file = "/Users/david/Documents/MemoryNetwork/output_lua/data.t7"
    voc = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7")
    index = torch.load("/Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index")
    x, y,f = data_loader.text_to_tensor(input_file,out_vocab_file,out_tensor_file)
end

saving /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7	


saving its index in /Users/david/Documents/MemoryNetwork/output_lua/vocab.t7_index	


putting sample tensor into /Users/david/Documents/MemoryNetwork/output_lua/sample.t7...	


saving /Users/david/Documents/MemoryNetwork/output_lua/sample.t7	
putting label tensor into /Users/david/Documents/MemoryNetwork/output_lua/data.t7...	


saving /Users/david/Documents/MemoryNetwork/output_lua/data.t7	


In [3]:
-- Checking x
print(x:size())
-- Checking y
print(y:size())
-- Checking f
print(f:size())

 70
 58
[torch.LongStorage of size 2]

 70
 58
[torch.LongStorage of size 2]

 70
  9
[torch.LongStorage of size 2]



In [4]:
-- pre-requists

require 'nn';
require 'nngraph';
require 'models.inference_module'
require 'utils.OneHot'

# Memory Network

In [5]:
require 'math';
require 'nn';
require 'nngraph';

MemoryModule = {}

function MemoryModule.create_network(num_mem,longuest_sentence_size,debug) -- longest_sentence,voc_size,num_mem)
    local inputs = {}
    local outputs = {}
    table.insert(inputs, nn.Identity()())
    -- 
    local reshaped_input =  nn.Reshape(1,-1)(inputs[1])
    local mem_proj = nn.SplitTable(1)(reshaped_input):annotate{name="mem_proj"}
    -- Creating memory mapping :
    local mem_size =  math.floor(longuest_sentence_size/num_mem)
    if debug then print("num_mem = "..num_mem) print("mem_size = "..mem_size) end
    for i=1,num_mem do
        local mem_mapping = nn.NarrowTable(1+(i-1)*mem_size,mem_size)(mem_proj)
        table.insert(outputs, nn.JoinTable(1)(mem_mapping)) -- Joining to get a table of tensors
    end 
    return nn.gModule(inputs, outputs)
end

In [ ]:
RepresentationModule = {}

function RepresentationModule.create_network(vocab_size) 
    local inputs = {}
    local outputs = {}
    table.insert(inputs, nn.Identity()())
    -- 
    local one_hot = OneHot.new(vocab_size)(inputs[1]) -- One Hot Encoding sub
    table.insert(outputs, one_hot)
    return nn.gModule(inputs, outputs)
end

In [ ]:
MixingGraph = {}

function MixingGraph.create_network()
    local inputs = {}
    local outputs = {}
    table.insert(inputs, nn.Identity()())
    table.insert(inputs, nn.Identity()())
    local repr_x = nn.Reshape(1,-1,false)(nn.Sum(1)(inputs[1]))
    local repr_y = nn.Reshape(1,-1,false)(nn.Sum(1)(inputs[2]))
    local sum = nn.CAddTable(1)({repr_x,repr_y})
    table.insert(outputs, sum)
    return nn.gModule(inputs,outputs)
end

local ee = torch.DoubleTensor(1,10):fill(1)
local rr = torch.DoubleTensor(1,10):fill(4)
-- print(nn.CAddTable(2):forward{ee,rr})

local m = MixingGraph.create_network()
print(m:forward{ee,rr})

In [201]:
local m = MixingGraph.create_network()

local rep = RepresentationModule.create_network(VOCAB_SIZE)
local xrepr = rep:forward(x[1])
local yrepr = rep:forward(y[1])

--print(nn.Reshape(1,-1,false):forward(nn.Sum(1):forward(yrepr)))
print(m:forward{xrepr,yrepr})

Columns 1 to 16
 106    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0

Columns 17 to 32
   0    0    0    0    2    0    0    0    0    0    0    0    0    0    0    2

Columns 33 to 48
   0    0    0    0    0    2    0    0    0    2    0    0    0    0    0    0

Columns 49 to 58
   2    0    0    0    0    0    0    0    0    0
[torch.DoubleTensor of size 1x58]



In [ ]:
NUM_MEM = 10
VOCAB_SIZE = 58
N_ITERATIONS = 1000
LEARNING_RATE = 1e-5

--------------  Network Creation

-- Incoding input
local rep = RepresentationModule.create_network(VOCAB_SIZE)
-- print(rep:forward(x[1]):size())
-- Setting memory
mem_mod = MemoryModule.create_network(NUM_MEM,VOCAB_SIZE)
-- Infering on memory 
o_mod = InferenceGraph.create_network(VOCAB_SIZE,3*VOCAB_SIZE)
-- Inferinf on Response
r_mod = InferenceGraph.create_network(VOCAB_SIZE,3*VOCAB_SIZE)

criterion = nn.MarginRankingCriterion(0.1)

--------------  Support Function

function print_memory_table(x_ind,fact,num_mem)
    local sup_memory = {}
    local temp1 = -1
    for i=1,f[1]:size(1) do
        local temp = math.floor(fact[x_ind][i]/num_mem)
        if temp ~= temp1 and temp ~= 0 then 
            table.insert(sup_memory,temp)
            temp1 = temp
        end
    end
    return sup_memory
end

function get_facts_memory(x_ind,fact,num_mem)
    local sup_memory = {}
    local temp1 = -1
    for i=1,f[1]:size(1) do
        local temp = math.floor(fact[x_ind][i]/num_mem)
        if temp ~= temp1 and temp ~= 0 then 
            table.insert(sup_memory,temp)
            temp1 = temp
        end
    end
    return sup_memory
end

function gradUpdate(mlp, x, memory, criterion, learningRate)
   local pred = mlp:forward(x)
   local err = criterion:forward(pred, memory)
   local gradCriterion = criterion:backward(pred, memory)
   mlp:zeroGradParameters()
   mlp:backward(x, gradCriterion)
   mlp:updateParameters(learningRate)
end

-------------- 



for i=1,N_ITERATIONS do
    local indice = torch.random(1,x:size(1))-- picking up a sentence in the training set
    local x_i, y_i = x[indice], y[indice]
    local m_i = get_facts_memory(indice,f,NUM_MEM)
    
    -- Going Through the Network 
    local xrepr = rep:forward(x_i)
    local yrepr = rep:forward(y_i)
    local a = mem_mod:forward(xrepr) -- a is the table of memories

    --
    local j = torch.random(1,NUM_MEM) -- picking up a memory
    local o_label = o_mod:forward{xrepr, a[m_i[1]]}
    if m_i[1] == j then
        -- resample if the sampling return the same memory as the supporting one
    else 
        print(a[m_i[1]])
        print(a[j])
        local o_score = o_mod:forward{xrepr,a[j]}
        print(o_label)
        print(o_score)
        local resized_pred = nn.JoinTable(1):forward{o_label, o_score}
        local err = criterion:forward(resized_pred,1)
        local gradCriterion = criterion:backward(resized_pred, 1)
        print("gradCriterion")
        local crit_false = nn.Reshape(1,1,false):forward(gradCriterion[2])
        local crit_true = nn.Reshape(1,1,false):forward(gradCriterion[1])
        o_mod:zeroGradParameters()
        o_mod:backward({xrepr,a[j]},crit_false)
        o_mod:backward({xrepr,m_i[1]},crit_true)
    end
end

print("Answer ")

-- local score_2 = r_mod:forward{xrepr,yrepr}
-- print(score_2)

## Debugging 

In [ ]:
for k,v in pairs(o_mod.forwardnodes) do 
    if  v.data.annotations.name == "inference_product" then -- v.data.annotations.name == "inference_proj_1" or v.data.annotations.name == "inference_proj_2" or
        print(v.data.module.gradInput[2]:fill(torch.random))
    end
end

In [506]:
print(get_facts_memory(10,f,10)[1])
--print(f[10])

2	


In [259]:
local ii = 

print(ii[2][1])

-4.3324330497092e-311	


In [82]:
function get_supporting_facts(indice, fact_tensor,entry_table)
    for i=1,fact_tensor:size(2) do 
        local temp = fact_tensor[indice][i]
        if temp ~= 0 then
            print(i)
            print(index[entry_table[indice][temp]])
        end
    end
end

get_supporting_facts(10,f,x)

1	
stereo	
2	
in	
3	
the	
4	
landing.	
5	


caden	
6	
joined	
7	
carmen	


In [ ]:
function get_supporting_memories(indice,facts,num_mem)
    

end

In [86]:
x:size(2)

58	


In [87]:
math.floor(longuest_sentence_size/num)

[string "local f = function() return math.floor(longue..."]:1: attempt to perform arithmetic on global 'longuest_sentence_size' (a nil value)
stack traceback:
	[string "local f = function() return math.floor(longue..."]:1: in function 'f'
	[string "local f = function() return math.floor(longue..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:210: in function </Users/david/torch/install/share/lua/5.1/itorch/main.lua:174>
	/Users/david/torch/install/share/lua/5.1/lzmq/poller.lua:80: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/david/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/david/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0105cbaa10: 

## Implementation of the Loss Function

In [105]:
function gradUpdate(mlp, x, y, criterion, learningRate)
   local pred = mlp:forward(x)
   local err = criterion:forward(pred, y)
   local gradCriterion = criterion:backward(pred, y)
   mlp:zeroGradParameters()
   mlp:backward(x, gradCriterion)
   mlp:updateParameters(learningRate)
end


local xrepr = o:forward(x[1])
gradUpdate(mlpa, {{x, y}, {x, z}}, 1, crit, 0.01)

Columns 1 to 26
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Columns 27 to 52
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

Columns 53 to 58
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0  0  0  0  0  0
 0

# Training the model